In [1]:
import sys
import os
from scipy import linalg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
sys.path.append("..")
# Enable interactive plotting in Jupyter Notebook

from utils.detection_utils import *

In [5]:
run_name = "RUN4"
n_stickers = 6
file_number = 10
test_file = "10mm2.mp4"

spiegel_calib = "spiegel.mp4"
front_calib = "front.mp4"
stereo_calib = "stereo.mp4"

In [6]:
calibration_dir = f"../videos/{run_name}/calibration"
detection_dir = f"../videos/{run_name}/detection"
detection_files = os.listdir(detection_dir)
files = os.listdir(f"../videos/{run_name}/detection")
model = YOLO("../weights/best.pt")
if test_file is None:
    test_file = files[file_number]

sC = stereoCamera()
sC.load_from_yaml(f"{run_name}.yaml")
vL = videoLoader()
Dt1 = ballDetector()   
Dt2 = ballDetector()
sDt1 = stickerDetector(n_stickers)
sDt2 = stickerDetector(n_stickers)

print("File= ", test_file)

File=  10mm2.mp4


In [ ]:


def draw_detections(img, detection_results, stickercoords=None):
    detection_results = detection_results[0]  
    boxes = detection_results.boxes.xyxy.tolist()
    classes = detection_results.boxes.cls
    for cl, box in zip(classes, boxes):
        if  cl == 0:  # 0 = Ball, 1 = Red Sticker
            img = cv2.rectangle(img, np.array(box[:2], dtype=int), np.array(box[2:], dtype=int), [200, 200, 200], 4)
        if cl == 1:
            img = cv2.rectangle(img, np.array(box[:2], dtype=int), np.array(box[2:], dtype=int), [0, 0, 200], 4)

    if stickercoords is not None:
        for n, coord in stickercoords.items():
            print("Key", n, "Coord", coord)
            cv2.putText(img, str(n), np.array(coord, dtype=int), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 255, 0), 1)
    return img

vL.load_video(f"../videos/{run_name}/detection/{test_file}")
frames = vL[30:-30]
stereo_coords = [[],[]]
sticker_stereo_cords = [[], []]
frame_count = []
for j, frame in enumerate(frames):
    frame1, frame2 = sC(frame)
    #frame1 = sC.undistort_image(frame1, 0)
    #frame2 = sC.undistort_image(frame2, 1)
    preds1 = model(frame1)
    preds2 = model(frame2)
    ballcoord1 = Dt1(preds1)
    ballcoord2 = Dt2(preds2)
    stickercoords1 = sDt1(preds1, mirror=True)
    stickercoords2 = sDt2(preds2)
        
    frame1 = draw_detections(frame1, preds1, stickercoords1)
    frame2 = draw_detections(frame2, preds2, stickercoords2)
    cv2.imshow("f1", frame1)
    cv2.imshow("f2", frame2)
    cv2.waitKey(1)
    if ballcoord1 is None or ballcoord2 is None or stickercoords1 is None or stickercoords2 is None:
        continue
    if len([k for k in stickercoords1.keys() if k in stickercoords2.keys()]) < 3 :
        continue
    
    stereo_coords[0].append(ballcoord1)
    stereo_coords[1].append(ballcoord2)
    sticker_stereo_cords[0].append(stickercoords1)
    sticker_stereo_cords[1].append(stickercoords2)
    frame_count.append(j)

cv2.destroyAllWindows()


0: 320x640 8 Stickers, 144.9ms
Speed: 13.1ms preprocess, 144.9ms inference, 583.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 Ball, 7 Stickers, 92.4ms
Speed: 3.2ms preprocess, 92.4ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)
Warming up 0
Warming up 0
First frame detection had 8 stickers detected, the real number of stickers should be 6, skipping!
First frame detection had 7 stickers detected, the real number of stickers should be 6, skipping!

0: 320x640 7 Stickers, 189.4ms
Speed: 4.3ms preprocess, 189.4ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 Ball, 7 Stickers, 172.6ms
Speed: 11.5ms preprocess, 172.6ms inference, 1.9ms postprocess per image at shape (1, 3, 320, 640)
Warming up 0
Warming up 1
First frame detection had 7 stickers detected, the real number of stickers should be 6, skipping!
First frame detection had 7 stickers detected, the real number of stickers should be 6, skipping!

0: 320x640 7 Stic

In [ ]:
print(np.array(stereo_coords).shape)
print(len(frame_count))    

In [ ]:

def triangulate(SC, point1, point2):
    RT1 = np.concatenate([np.eye(3), [[0], [0], [0]]], axis=-1)
    RT2 = np.concatenate([SC.conf["rotation_matrix"][0], SC.conf["translation_matrix"][0]], axis=-1)

    P1 = SC.conf["camera_matrix"][0] @ RT1
    P2 = SC.conf["camera_matrix"][1] @ RT2

    cm0 = np.array(SC.conf["camera_matrix"][0])
    cm1 = np.array(SC.conf["camera_matrix"][1])
    dist0 = np.array(SC.conf["distortion"][0])
    dist1 = np.array(SC.conf["distortion"][0])
 
    point1 = cv2.undistortPoints(point1, cm0, dist0 , None, cm0)[0][0]
    point2 = cv2.undistortPoints(point2, cm1, dist1, None, cm1)[0][0]

    coordinate = cv2.triangulatePoints(P1, P2, point1, point2)
    coordinate = (coordinate[:3, :] / coordinate[3, :]).T[0]

    return coordinate


def compute_transform_matrix(points_A, points_B):
    """Calculates the transform matrix to transform any point of one coordinatesystem (3d) to another"""
    # Assuming points_A and points_B are numpy arrays with shape (N, 3)
    # where N is the number of points (N >= 3),
    # Reshaping the points
    points_A = np.array(points_A)
    points_B = np.array(points_B)
    points_A = points_A.T
    points_B = points_B.T

    # Calculate the centroids of both point sets
    centroid_A = np.mean(points_A, axis=1, keepdims=True)
    centroid_B = np.mean(points_B, axis=1, keepdims=True)

    # Compute the centered point sets
    centered_A = points_A - centroid_A
    centered_B = points_B - centroid_B

    # Compute the covariance matrix
    covariance_matrix = centered_A @ centered_B.T

    # Perform Singular Value Decomposition
    U, _, Vt = np.linalg.svd(covariance_matrix)

    # Calculate the rotation matrix
    rotation_matrix = Vt.T @ U.T

    # Calculate the translation vector
    translation_vector = centroid_B - rotation_matrix @ centroid_A

    # Create the transformation matrix
    transform_matrix = np.eye(4)
    transform_matrix[:3, :3] = rotation_matrix
    transform_matrix[:3, 3] = translation_vector.flatten()
    return transform_matrix

def transform_point(point, transform_matrix):
    """Transforms a point to a nother coordinate system (3d) based on a transform_matrix"""
    # point needs one extra dimension
    point = np.array([point])
    point_homogeneous = np.hstack((point, np.ones((1, 1))))
    new_point_homogeneous = transform_matrix @ point_homogeneous.T

    # Extract the transformed point in system B
    new_point = new_point_homogeneous[:3, :].T
    return new_point[0]

# Calculating the 3D Points

In [ ]:
ball3d = [triangulate(sC, point1, point2) for point1, point2 in zip(stereo_coords[0], stereo_coords[1])]
sticker3d = []

for sticker_coords1, sticker_coords2 in zip(*sticker_stereo_cords):
    valid_points = [k for k in sticker_coords1.keys() & sticker_coords2.keys()]
    points = {k: triangulate(sC, sticker_coords1[k], sticker_coords2[k]) for k in valid_points}
    sticker3d.append(points)

In [ ]:
# Removing the yar movement
bad_stickers = [0, 1]

fixedball3d = [ball3d[0]]
base_sticker_points = sticker3d[0]
fixedsticker3d = [[v for v in base_sticker_points.values()]]

for ball_point, sticker_points, in zip(ball3d[1:], sticker3d[1:]):
    valid_points = [k for k in base_sticker_points.keys() & sticker_points.keys()]
    valid_points = [k for k in valid_points if k not in bad_stickers] # ignoring stickers that had bad detection, evaluate yourself
    transform_matrix = compute_transform_matrix([sticker_points[k] for k in valid_points], [base_sticker_points[k] for k in valid_points])
    ball_point_new = transform_point(ball_point, transform_matrix)
    sticker_point_new = transform_point(sticker_points[0], transform_matrix)
    #print("Diff", ball_point -ball_point_new)
    fixedball3d.append(ball_point_new)
    fixedsticker3d.append([transform_point(p, transform_matrix) for p in sticker_points.values()])
    

# Calculating the sticker distances

In [ ]:
def avg_dist(stickercoords, key1, key2):
    dists = []
    for s in stickercoords:
        try: 
            coords1 = s[key1]
            coords2 = s[key2]
        except KeyError:
            continue
        diff = coords1 - coords2
        dist = np.linalg.norm(diff)
        dists.append(dist)
    return np.array(dists).mean()

cv2.imshow("f1", frame1)
cv2.imshow("f2", frame2)
cv2.waitKey(0)

In [ ]:
avg_dist(sticker3d, 2, 5)/0.0695, avg_dist(sticker3d, 4, 3)/0.039, avg_dist(sticker3d, 2, 3)/0.052, avg_dist(sticker3d, 5, 4)/0.049

# Plotting the detections

In [ ]:

coords = np.array(ball3d)  # 
coords_sticker = np.array([x for p in sticker3d for x in p.values()]) # extracting the sticker coordinates
# Create line connections between sequential points
connections = [[a, b] for a, b in zip(range(len(coords)-1), range(1, len(coords)))]
coords = np.concatenate([coords, coords_sticker], 0)
print(coords.shape)
# Calculate max range to set equal axes
x_range = [coords[:, 0].min(), coords[:, 0].max()]
y_range = [coords[:, 1].min(), coords[:, 1].max()]
z_range = [coords[:, 2].min(), coords[:, 2].max()]

# Find the maximum range
max_range = max(x_range[1] - x_range[0], y_range[1] - y_range[0], z_range[1] - z_range[0])

# Calculate the mid points for each axis
x_mid = sum(x_range) / 2
y_mid = sum(y_range) / 2
z_mid = sum(z_range) / 2

# Set the range for each axis to be the max range centered around the mid point
x_range = [x_mid - max_range / 2, x_mid + max_range / 2]
y_range = [y_mid - max_range / 2, y_mid + max_range / 2]
z_range = [z_mid - max_range / 2, z_mid + max_range / 2]

# Create the figure object
fig = go.Figure()

# Add lines between connected points
for conn in connections:
    fig.add_trace(go.Scatter3d(
        x=[coords[conn[0], 0], coords[conn[1], 0]],
        y=[coords[conn[0], 1], coords[conn[1], 1]],
        z=[coords[conn[0], 2], coords[conn[1], 2]],
        mode='lines',
        line=dict(color='blue', width=2),
        name='Line'
    ))

# Add markers for each point
fig.add_trace(go.Scatter3d(
    x=coords[:, 0],
    y=coords[:, 1],
    z=coords[:, 2],
    mode='markers',
    marker=dict(size=1, color='red'),
    name='Points'
))

# Update plot appearance
fig.update_layout(
    title="3D Line Plot",
    scene=dict(
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        zaxis_title='Z Coordinate',
        xaxis=dict(range=x_range, autorange=False),
        yaxis=dict(range=y_range, autorange=False),
        zaxis=dict(range=z_range, autorange=False),
        aspectmode='manual'
        #,
        #aspectratio=dict(x=1, y=1, z=1)
    )
)

# Show the plot
fig.show()

In [ ]:
coords = np.array(fixedball3d)  # Update this line based on your actual data
coords_sticker = np.array([x for p in fixedsticker3d for x in p])

# Create line connections between sequential points
connections = [[a, b] for a, b in zip(range(len(coords)-1), range(1, len(coords)))]
coords = np.concatenate([coords, coords_sticker], 0)
# Calculate max range to set equal axes
x_range = [coords[:, 0].min(), coords[:, 0].max()]
y_range = [coords[:, 1].min(), coords[:, 1].max()]
z_range = [coords[:, 2].min(), coords[:, 2].max()]

# Find the maximum range
max_range = max(x_range[1] - x_range[0], y_range[1] - y_range[0], z_range[1] - z_range[0])

# Calculate the mid points for each axis
x_mid = sum(x_range) / 2
y_mid = sum(y_range) / 2
z_mid = sum(z_range) / 2

# Set the range for each axis to be the max range centered around the mid point
x_range = [x_mid - max_range / 2, x_mid + max_range / 2]
y_range = [y_mid - max_range / 2, y_mid + max_range / 2]
z_range = [z_mid - max_range / 2, z_mid + max_range / 2]

# Create the figure object
fig = go.Figure()

# Add lines between connected points
for conn in connections:
    fig.add_trace(go.Scatter3d(
        x=[coords[conn[0], 0], coords[conn[1], 0]],
        y=[coords[conn[0], 1], coords[conn[1], 1]],
        z=[coords[conn[0], 2], coords[conn[1], 2]],
        mode='lines',
        line=dict(color='blue', width=2),
        name='Line'
    ))

# Add markers for each point
fig.add_trace(go.Scatter3d(
    x=coords[:, 0],
    y=coords[:, 1],
    z=coords[:, 2],
    mode='markers',
    marker=dict(size=1, color='red'),
    name='Points'
))

# Update plot appearance
fig.update_layout(
    title="3D Line Plot",
    scene=dict(
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        zaxis_title='Z Coordinate',
        xaxis=dict(range=x_range, autorange=False),
        yaxis=dict(range=y_range, autorange=False),
        zaxis=dict(range=z_range, autorange=False),
        aspectmode='manual'
        ,
        aspectratio=dict(x=1, y=1, z=1)
    )
)

# Show the plot
fig.show()

In [ ]:
x = np.array([0.01499442, -0.04779105, 0.61447])
y = np.array([-0.03845, -0.04608, 0.61111])

diff = x -y

print(diff)
dist = (diff[0]**2 + diff[1]**2 + diff[2]**2)**0.5
dist

# testing stereo vision results 


In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np

coords = np.array(ball3d)  # Update this line based on your actual data

# Calculate max range to set equal axes
x_range = [coords[:, 0].min(), coords[:, 0].max()]
y_range = [coords[:, 1].min(), coords[:, 1].max()]
z_range = [coords[:, 2].min(), coords[:, 2].max()]

# Find the maximum range
max_range = max(x_range[1] - x_range[0], y_range[1] - y_range[0], z_range[1] - z_range[0])

# Calculate the mid points for each axis
x_mid = sum(x_range) / 2
y_mid = sum(y_range) / 2
z_mid = sum(z_range) / 2

# Set the range for each axis to be the max range centered around the mid point
x_range = [x_mid - max_range / 2, x_mid + max_range / 2]
y_range = [y_mid - max_range / 2, y_mid + max_range / 2]
z_range = [z_mid - max_range / 2, z_mid + max_range / 2]
frame_numbers = np.array(frame_count)
frame_numbers -= frame_numbers[0]

# Calculate total frames for 30 fps over the maximum frame index
total_frames = max(frame_numbers) + 1  # subtracting warmupsteps

# Create interpolated coordinates array
interpolated_coords = np.zeros((total_frames, 3))


for i in range(len(frame_numbers) - 1):
    start_frame, end_frame = frame_numbers[i], frame_numbers[i + 1]
    start_coords, end_coords = coords[i], coords[i + 1]
    for frame in range(start_frame, end_frame + 1):
        #print(frame, (1 - t) * start_coords + t * end_coords)
        t = (frame - start_frame) / (end_frame - start_frame)
        interpolated_coords[frame] = (1 - t) * start_coords + t * end_coords


print(interpolated_coords[:,0].min(), interpolated_coords[:,0].max())
print(interpolated_coords[:,1].min(), interpolated_coords[:,1].max())
print(interpolated_coords[:,2].min(), interpolated_coords[:,2].max())

# Using the interpolated coordinates for the animation
fig = go.Figure(
    data=[go.Scatter3d(
        x=[interpolated_coords[0, 0]], 
        y=[interpolated_coords[0, 1]], 
        z=[interpolated_coords[0, 2]],
        mode='markers',
        marker=dict(size=4, color='red')  # Adjust the size for visibility
    )],
    layout=go.Layout(
        scene=dict(
            xaxis=dict(range=[x_range[0], x_range[1]]),
            yaxis=dict(range=[y_range[0], y_range[1]]),
            zaxis=dict(range=[z_range[0], z_range[1]]),
            aspectmode='cube'  # Ensures all axes have the same scale
        ),
        updatemenus=[dict(
            type='buttons',
            showactive=False,
            buttons=[{'label': 'Play',
                      'method': 'animate',
                      'args': [None, {"frame": {"duration": 50, "redraw": True},
                                      "fromcurrent": True, "transition": {"duration": 30, "easing": "linear"}}]}]
        )]
    ),
    frames=[
        go.Frame(data=[go.Scatter3d(
            x=[coord[0]], y=[coord[1]], z=[coord[2]],
            mode='markers',
            marker=dict(size=4, color='red')  # Adjust the size to maintain the diameter of the ball
        )]) for coord in interpolated_coords
    ]
)

fig.show()

In [ ]:
coords3d
